In [ ]:
!pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -q transformers
#Downloading pre_trained Sentiment analyser from HuggingFace Library
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["very Good", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9998520612716675},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [ ]:
chrome_reviews = pd.read_csv('/content/chrome_reviews.csv')
chrome_reviews.shape

(7204, 10)

In [ ]:
chrome_reviews = chrome_reviews.loc[:, ['ID','Text','Star','User Name', 'Thumbs Up','Review URL','Developer Reply', 'Version', 'Review Date', 'App ID']]
chrome_reviews.columns

Index(['ID', 'Text', 'Star', 'User Name', 'Thumbs Up', 'Review URL',
       'Developer Reply', 'Version', 'Review Date', 'App ID'],
      dtype='object')

In [ ]:
chrome_reviews.head()

,ID,Text,Star,User Name,Thumbs Up,Review URL,Developer Reply,Version,Review Date,App ID
0,3886,This is very helpfull aap.,5,INDIAN Knowledge,0,https://play.google.com/store/apps/details?id=...,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,Good,3,Ijeoma Happiness,2,https://play.google.com/store/apps/details?id=...,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,Not able to update. Neither able to uninstall.,1,Priti D BtCFs-29,0,https://play.google.com/store/apps/details?id=...,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,Nice app,4,Ajeet Raja,0,https://play.google.com/store/apps/details?id=...,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,Many unwanted ads,1,Rams Mp,0,https://play.google.com/store/apps/details?id=...,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [ ]:
chrome_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7204 entries, 0 to 7203
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               7204 non-null   int64 
 1   Review URL       7204 non-null   object
 2   Text             7203 non-null   object
 3   Star             7204 non-null   int64 
 4   Thumbs Up        7204 non-null   int64 
 5   User Name        7204 non-null   object
 6   Developer Reply  95 non-null     object
 7   Version          7119 non-null   object
 8   Review Date      7204 non-null   object
 9   App ID           7204 non-null   object
dtypes: int64(3), object(7)
memory usage: 562.9+ KB


In [ ]:
import re
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

# Function for converting emojis into word
def convert_emojis(text):
    text = str(text)
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text


def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',str(text))

#Cleaing Special Characters
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem


def to_lower(text):
    return text.lower()


In [ ]:
#Cleaning Text of Emojis, Speacial Characters
chrome_reviews['Text'].dropna()
chrome_reviews['Tokenised_Text'] = chrome_reviews['Text'].apply(convert_emojis)
chrome_reviews['Tokenised_Text'] = chrome_reviews['Text'].apply(clean)
chrome_reviews['Tokenised_Text'] = chrome_reviews['Tokenised_Text'].apply(is_special)
chrome_reviews['Tokenised_Text'] = chrome_reviews['Tokenised_Text'].apply(to_lower)

In [ ]:
eo = chrome_reviews['Text'][56]
eo

'👎'

In [ ]:
to_lower(is_special(clean(convert_emojis(eo))))


'thumbs down'

In [ ]:
chrome_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7204 entries, 0 to 7203
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               7204 non-null   int64 
 1   Review URL       7204 non-null   object
 2   Text             7203 non-null   object
 3   Star             7204 non-null   int64 
 4   Thumbs Up        7204 non-null   int64 
 5   User Name        7204 non-null   object
 6   Developer Reply  95 non-null     object
 7   Version          7119 non-null   object
 8   Review Date      7204 non-null   object
 9   App ID           7204 non-null   object
 10  Tokenised_Text   7204 non-null   object
dtypes: int64(3), object(8)
memory usage: 619.2+ KB


In [ ]:
y = sentiment_pipeline(chrome_reviews['Tokenised_Text'][4])
y

[{'label': 'NEGATIVE', 'score': 0.9996932744979858}]

In [ ]:
#Helppr Function to get sentiments
def get_sentiments(text):
  sentiments = sentiment_pipeline(text)
  return sentiments[0]['label']


In [ ]:
%%time
get_sentiments(chrome_reviews['Text'][4])

CPU times: user 42.6 ms, sys: 0 ns, total: 42.6 ms
Wall time: 69 ms


'NEGATIVE'

In [ ]:
#Labeling Text as Either Positive or Negative Based on  Sentiments
chrome_reviews['user_sentiments'] = chrome_reviews['Tokenised_Text'].apply(get_sentiments)

In [ ]:
chrome_reviews.tail()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID,Tokenised_Text,user_sentiments
7199,684987,https://play.google.com/store/apps/details?id=...,Bagusss..,5,0,boima panjaitan,NaN,88.0.4324.93,2021-03-05,com.android.chrome,bagusss..,NEGATIVE
7200,684988,https://play.google.com/store/apps/details?id=...,Bad version 😔,1,0,निशान्त सिंह,NaN,88.0.4324.181,2021-03-05,com.android.chrome,bad version 😔,NEGATIVE
7201,684989,https://play.google.com/store/apps/details?id=...,One thing that I have to say I can't spelled t...,5,0,Virgie Allen,NaN,89.0.4389.72,2021-03-05,com.android.chrome,one thing that i have to say i can't spelled t...,POSITIVE
7202,684990,https://play.google.com/store/apps/details?id=...,Excellent,5,0,Kazuo Guevarra,NaN,89.0.4389.72,2021-03-05,com.android.chrome,excellent,POSITIVE
7203,684991,https://play.google.com/store/apps/details?id=...,After update it lag and always slow same goes ...,1,0,Hanin Amni,NaN,88.0.4324.93,2021-03-05,com.android.chrome,after update it lag and always slow same goes ...,NEGATIVE


In [ ]:
chrome_reviews.to_csv('Output.csv')

In [ ]:
chrome_reviews_positive = chrome_reviews[chrome_reviews.user_sentiments == 'POSITIVE']
chrome_reviews_positive.shape

(4201, 12)

In [ ]:
#Holding  3 star rating as a thresold, Showing reviews with Positive Sentiments and Low Ratings
low_rating = chrome_reviews_positive[chrome_reviews_positive.Star < 3]
low_rating.shape

(329, 12)

In [ ]:
low_rating[['Text','user_sentiments','Star','Tokenised_Text']]

,Text,user_sentiments,Star,Tokenised_Text
15,Very coming in real status. Thank you,POSITIVE,1,very coming in real status. thank you
42,Okk kind but bad then brave,POSITIVE,1,okk kind but bad then brave
84,App updated issue,POSITIVE,1,app updated issue
91,Jelek..........?!!!!!!!!!!!!!!!!!,POSITIVE,1,jelek..........?!!!!!!!!!!!!!!!!!
101,Good,POSITIVE,1,good
...,...,...,...,...
7051,Helpfull,POSITIVE,1,helpfull
7076,Second,POSITIVE,2,second
7101,Wirst,POSITIVE,1,wirst
7113,Idk,POSITIVE,1,idk
